In [1]:
import os
import dask.dataframe as dd
import pandas as pd
from pathlib import Path
import ml_functions

2024-09-21 12:37:50.200641: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-09-21 12:37:50.260074: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [8]:
TMPDIR = os.getenv("TMPDIR")

old = pd.read_parquet(Path(TMPDIR) / f"HRRR.4hr.par")
new = pd.read_parquet(Path(TMPDIR) / f"HRRR.4hr.par_hide")

In [9]:
old.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 164248705 entries, (0.0, 0.0, Timestamp('2020-11-30 00:00:00'), Timestamp('2020-12-02 00:00:00'), 12, 48, 48, 26.24666404724121, -98.2596664428711) to (nan, nan, nan, Timestamp('2022-01-01 00:00:00'), 38, 72, 24, 43.88113883653136, -75.8128258641754)
Data columns (total 16 columns):
 #   Column                       Dtype  
---  ------                       -----  
 0   (y_pred, cg_20km_4hr)        float32
 1   (y_pred, ic_20km_4hr)        float32
 2   (y_pred, cg.ic_20km_4hr)     float32
 3   (y_pred, flashes_20km_4hr)   float32
 4   (y_pred, cg_40km_4hr)        float32
 5   (y_pred, ic_40km_4hr)        float32
 6   (y_pred, cg.ic_40km_4hr)     float32
 7   (y_pred, flashes_40km_4hr)   float32
 8   (y_label, cg_20km_4hr)       float32
 9   (y_label, ic_20km_4hr)       float32
 10  (y_label, cg.ic_20km_4hr)    float32
 11  (y_label, flashes_20km_4hr)  float32
 12  (y_label, cg_40km_4hr)       float32
 13  (y_label, ic_40km_4hr)       fl

In [10]:
new.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 120663000 entries, (0, 0, Timestamp('2020-11-30 00:00:00'), Timestamp('2020-12-02 00:00:00'), 12, 48, 48, 26.246664, -98.25967) to (0, 4, Timestamp('2021-12-31 00:00:00'), Timestamp('2021-12-31 23:00:00'), 46, 29, 23, 48.911613, -118.32521)
Data columns (total 16 columns):
 #   Column                       Dtype  
---  ------                       -----  
 0   (y_pred, cg_20km_4hr)        float32
 1   (y_pred, ic_20km_4hr)        float32
 2   (y_pred, cg.ic_20km_4hr)     float32
 3   (y_pred, flashes_20km_4hr)   float32
 4   (y_pred, cg_40km_4hr)        float32
 5   (y_pred, ic_40km_4hr)        float32
 6   (y_pred, cg.ic_40km_4hr)     float32
 7   (y_pred, flashes_40km_4hr)   float32
 8   (y_label, cg_20km_4hr)       float32
 9   (y_label, ic_20km_4hr)       float32
 10  (y_label, cg.ic_20km_4hr)    float32
 11  (y_label, flashes_20km_4hr)  float32
 12  (y_label, cg_40km_4hr)       float32
 13  (y_label, ic_40km_4hr)       float32
 14 

In [17]:
old.compare(new.reindex(old.index))

fold,fit,initialization_time,valid_time,y,x,forecast_hour,lat,lon


In [11]:
old2 = old.set_index(["initialization_time","forecast_hour"], append=True)
old2.compare(new.set_index(["initialization_time","forecast_hour"], append=True).reindex(old2.index))

KeyError: "None of ['initialization_time', 'forecast_hour'] are in the columns"

In [6]:
new.tail().iloc[:,-8:] >= 10

,cg_20km_4hr,ic_20km_4hr,cg.ic_20km_4hr,cg_40km_4hr,ic_40km_4hr,cg.ic_40km_4hr,flashes_20km_4hr,flashes_40km_4hr
pt,,,,,,,,
1304,False,False,False,False,False,False,False,False
1305,False,False,False,False,False,False,False,False
1306,False,False,False,False,False,False,False,False
1307,False,False,False,False,False,False,False,False
1308,False,False,False,False,False,False,False,False


In [7]:
new = ml_functions.rptdist2bool(new, args)

NameError: name 'args' is not defined

In [ ]:
old.loc[(slice(None,"20201104T04"), slice(None), slice(None), slice(None))]

In [ ]:
old.columns[-40:]

In [ ]:
new[(new.valid_time <= pd.to_datetime("20201104T04"))].set_index(["valid_time","y","x","forecast_hour"])

In [ ]:
df = (
    new[
        (new.valid_time <= pd.to_datetime("20201104T04"))
    ].set_index(["valid_time","y","x","forecast_hour"])
    - old.loc[(slice(None,"2020-11-04 04:00:00"), slice(None), slice(None), slice(None))]
)
df = df.drop(columns={f"cg_20km_{twin}hr", f"flashes_20km_{twin}hr", f"ic_20km_{twin}hr"}).dropna()
x = df.loc[:, (df != 0).any(axis=0)].drop(columns="initialization_time")
x

In [ ]:
x.loc[(x != 0).any(axis=1)]

In [ ]:
x.loc[(x != 0).any(axis=1)].idxmin()

In [ ]:
old.loc[("2020-08-25 16:00:00", 32, 66, 16),["lon","lat",f"ic_40km_{twin}hr"]]

In [ ]:
new.loc[(new.valid_time=="2020-08-25 16:00:00") & (new.y==32) & (new.x==66),["forecast_hour","lon","lat",f"ic_40km_{twin}hr"]]

In [ ]:
old = old.set_index("initialization_time", append=True)
old.index = old.index.reorder_levels(order=["initialization_time", "valid_time", "y", "x"])

In [ ]:
new = new.set_index(["initialization_time", "valid_time", "y", "x"])

In [ ]:
d=(new-old).dropna(subset="CAPESHEAR")

In [ ]:
d.head()

In [ ]:
parser = ml_functions.get_argparser()
args, argv = parser.parse_known_args()
args.batchnorm = True
args.model = 'HRRR'
args.epochs = 10
args.debug = False
args.kfold = 1
args.learning_rate = 0.01
args.neurons = [32,32]
args.optimizer = 'SGD'
args.suite = "with_storm_mode"
args.teststart = pd.to_datetime("19000101")
args.testend = pd.to_datetime("20220101")
args.labels = ['sighail_40km_2hr', 'sigwind_40km_2hr', 'hailone_40km_2hr', 'wind_40km_2hr', 'torn_40km_2hr', 'any_40km_2hr']

In [ ]:
new = ml_functions.rptdist2bool(new, args)


In [ ]:
new[["sighail_40km_2hr","sigwind_40km_2hr"]].groupby("initialization_time").mean().plot()

In [ ]:
new[["sighail_40km_2hr","sigwind_40km_2hr"]].groupby("initialization_time").mean().plot()

In [ ]:
rptdist = 40
lsrtypes = ["sighail", "sigwind", "hailone", "wind", "torn"]
oldtwin = [0,1,2]
label_cols = [f"{r}_rptdist_{t}hr" for r in lsrtypes for t in oldtwin]
# refer to new label names (with f"{rptdist}km" not f"rptdist")
new_label_cols = {r:r.replace("rptdist", f"{rptdist}km") for r in label_cols}
label_cols = list(new_label_cols.values())
new_label_cols

In [ ]:
old = old.rename(columns=new_label_cols, copy=False)
old[label_cols] = (0 <= old[label_cols]) & (old[label_cols] < rptdist)

In [ ]:
new = new.rename(columns=new_label_cols, copy=False)
new[label_cols] = (0 <= new[label_cols]) & (new[label_cols] < rptdist)

In [ ]:
xx = old.sigwind_40km_2hr.groupby(["y","x"]).mean()
xx.reset_index().plot(x="x",y="y", kind="scatter", c=xx)

In [ ]:
yy = new.sigwind_40km_2hr.groupby(["y","x"]).mean() - old.sigwind_40km_2hr.groupby(["y","x"]).mean()
yy.reset_index().plot(x="x",y="y", kind="scatter", c = yy, colormap="cool")

In [ ]:
sl = new.xs(slice("20210101","20211231T12"), level="initialization_time")

In [ ]:
sl.mean().tail(40)

In [ ]:
cr.tail(40)

In [ ]:
set(new.columns) - set(old.columns)

In [ ]:
set(old.columns) - set(new.columns)

In [ ]:
old.torn_40km_0hr.max()

In [ ]:
old.index.get_level_values("initialization_time").min(),old.index.get_level_values("initialization_time").max()

In [ ]:
new.index.get_level_values("initialization_time").min(),new.index.get_level_values("initialization_time").max()

In [ ]:
pd.to_datetime(new.index.get_level_values("initialization_time"))

In [ ]:
old.index.equals(new.index)

In [ ]:
all(new.index.get_level_values("initialization_time") == old.index.get_level_values("initialization_time"))

In [ ]:
np.array_equal(old.iloc[:,40:244].values , new.iloc[:,40:244].values, equal_nan=True )

In [ ]:
new.columns[224:]

In [ ]:
import numpy as np